# [Find k-th permu](https://leetcode.com/problems/permutation-sequence/)
- **Given**: `N`, `K`
    + A set size `N`: `[1, 2, 3, ..., N]`
- Find the `K-th` permuation of the set

#### Examples

```
// Input
N = 3, K = 3

// Output
213

// Explaination
1. "123"
2. "132"
3. "213"
4. "231"
5. "312"
6. "321"
```

```
Input: N = 4, K = 9
Output: "2314"
```

```
Input: N = 3, K = 1
Output: "123"
```


#### Bruteforce - O(n!) - TLE

```C++
class Solution {
private:
    vector<char> _seeds;
    vector<bool> _vis;

    string _cand;
    vector<string> all_ans;
public:
    void dfs(int idx) {
        if(idx == _seeds.size()) {
            all_ans.push_back(_cand);
            return;
        }
        for(int i=0; i<_seeds.size(); ++i) {
            if(_vis[i] == false) {
                _cand += _seeds[i];
                _vis[i] = true;

                dfs(idx + 1);

                _cand.pop_back();
                _vis[i] = false;
            }
        }
    }
    string getPermutation(int N, int K) {
        _vis.assign(N, false);
        for(int i=1; i<=N;++i) _seeds.push_back( i + '0' );

        dfs(0);
        return all_ans[K-1];
    }
};
```

#### Optimized bruteforce - O(K)
- Early stopping backtracking

```C++
class Solution {
private:
    vector<char> _seeds;
    vector<bool> _vis;

    int _k;
    string _cand = "";
    string _ans = "";
public:
    bool dfs(int idx) {
        if(idx == _seeds.size()) {
            _k -= 1;
            if(_k == 0) {
                // Early stopping dfs
                _ans = _cand;
                return true;
            }
            return false;
        }
        for(int i=0; i<_seeds.size(); ++i) {
            if(_vis[i] == false) {
                _cand += _seeds[i];
                _vis[i] = true;

                // Early stopping dfs
                if(dfs(idx + 1)) return true;

                _cand.pop_back();
                _vis[i] = false;
            }
        }
        return false;
    }
    string getPermutation(int N, int K) {
        _k = K;
        _vis.assign(N, false);
        for(int i=1; i<=N;++i) _seeds.push_back( i + '0' );

        dfs(0);
        return _ans;
    }
};
```

#### Factorial base number - $O(N^2)$
- From base 2 number (binary)

$$num_{\text{base 10}} = \sum\limits_{m = 0}^{N-1} k_m 2^m\ ,\ 0 \leq k_m \leq 1$$


- We can build a **Factorial base number**
$$num_{\text{base 10}} = \sum\limits_{m = 0}^{N-1} k_m m!\ ,\ 0 \leq k_m \leq m$$

<img src="./img/9.png" width="800"/>


```C++
class Solution {
public:
    string getPermutation(int N, int K) {
        // Cache fac = {1, 1, 2, 6, 24, ... n!}
        vector<int> fac(N+1, 1);
        for(int i=1; i<=N; ++i) {
            fac[i] = i*fac[i-1];
        }

        // Convert permu_num (base 10) -> factorial system representation (base m!)
        //      permu_num: permutation number (base 10): 0,1,2,3,4,5,6, etc.
        //      k_m: factorial system representation
        // Eg: permu_num = 1 -> k_m = [2]0 [1]1 [0]0
        // Eg: permu_num = 2 -> k_m = [2]1 [1]0 [0]0
        // Eg: permu_num = 3 -> k_m = [2]1 [1]1 [0]0
        int permu_num = K - 1;
        vector<int> k_m(N);
        for(int i=N-1; i>=0; --i) {
            k_m[i] = permu_num/fac[i];
            permu_num -= k_m[i]*fac[i];
        }

        // Generate set of N: {1,2,3}
        vector<int> seeds;
        for(int i=1; i<=N; ++i) {
            seeds.push_back(i);
        }

        // Interprete k_m -> permutation
        string ans = "";
        for(int i=N-1; i>=0; --i) {
            ans += to_string(seeds[k_m[i]]);
            seeds.erase(seeds.begin() + k_m[i]);
        }
        return ans;
    }
};
```